## Задание
Вам будет предоставлен набор данных о задачах, которые выполнялись в рамках строительства капитальных объектов на месторождениях нефти и газа. Набор содержит информацию о примерно 716 тысячах задач. Для каждой из задач доступна информация о ее названии в строительном плане, а также частично заданная информация об иерархии задач и обобщенных классах наименований, к которым относятся эти задачи (двух разных степеней детализации).

Используя эти данные, вам необходимо будет разработать семантическую модель, которая позволяла бы эффективно определять обобщенные классы для задач, у которых эта информация не представлена.

### Каждая задача описывается следующими атрибутами.
- work_name 
    * Текстовое название задачи в строительном плане (без предобработки).
- upper_works
    * Информация об иерархии названий объектов и блоков работ, в рамках которых выполнялась эта задача. Если задачи имеют одинаковое значение этого атрибута – это означает, что они выполнялись в рамках одного блока работ над одним объектом
(может быть пустым).
- generalized_work_class
    * Информация об обобщенном классе наименований работ, к
    которому относится задача (может быть пустым).
- global_work_class
    * Информация о самом высоком уровне обобщения названия задачи
(может быть пустым).


## Решение:

Можно заметить, что записи в train_data имеют четко опереденный вид
Те в один класс generalized_work_class входит строго определенный набор work_name
И также в класс global_work_class входит уникальная комбинация (work_name; generalized_work_class)

ТО ЕСТЬ НАШИ КЛАССЫ ЛИНЕЙНО РАЗДЕЛИМЫ

Именно поэтому пробует классифицировать тестовые данные именно по такому принципу. 

### Problem: 
1. work name написаны с ошибкой, то есть нужно сначала исправить написание, чтобы было все унифицированно

------------
Метрикой можно взять простую accuracy, поскольку мы не делаем классификацию ML методами, и важно чтобы максимальный объем данных был классифицирован семантически


In [79]:
import pandas as pd
import numpy as np

## 1. Get the data and do spell correction

In [93]:
# remove outliers 
df = pd.read_csv('./Checkpoints/lab2_oil_gas_field_construction_data.csv')

train_data = df.loc[df['generalized_work_class'].notna()]
test_data = pd.read_csv('./Checkpoints/lab2_test_dataset.csv', sep=';')

cnt = train_data.generalized_work_class.value_counts().to_frame().reset_index()
cnt.columns

Index(['generalized_work_class', 'count'], dtype='object')

In [94]:
test_data = test_data.loc[(test_data['work_name'].notna())&(test_data.generalized_work_class.notna())]

## 2. Correct spell mistakes 
- inify the form how the work name is written to use it for semantical class-tion later

In [95]:
from spellchecker import SpellChecker

russian = SpellChecker(language='ru', distance=1)

def correct_the_spelling(s):
    '''Corrects the spell mistakes in string'''
    new_string = ''
    for i in s:
        if i not in ['свай', 'балок', 'лестниц', 'т1', 'газа', 'бурение', 'пнр', 'мк']:
            i_corr = russian.correction(i)
            if i_corr != None:
                new_string = new_string + " " + i_corr
            else:
                new_string = new_string + " " + i
        else:
            new_string = new_string + " " + i
    return new_string


train_data['work_name_corr'] = train_data['work_name'].str.split()
train_data['work_name_corr'] = train_data['work_name_corr'].apply(correct_the_spelling)

test_data['work_name_corr'] = test_data['work_name'].str.split()
test_data['work_name_corr'] = test_data['work_name_corr'].apply(correct_the_spelling)

/var/folders/tt/rhnv8s9j45dfkl45r1y57vw80000gn/T/ipykernel_73742/938801483.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['work_name_corr'] = train_data['work_name'].str.split()
/var/folders/tt/rhnv8s9j45dfkl45r1y57vw80000gn/T/ipykernel_73742/938801483.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['work_name_corr'] = train_data['work_name_corr'].apply(correct_the_spelling)


## 3. If a test data work_name is in previously accured mapping

- Automatically classify it in a class of generalized_work_class

In [96]:
# in dict write {'generalized_work_class': 'unique_work_names'}

from pprint import pprint

target_text_dict = {}
unique_targets = train_data['generalized_work_class'].unique()

for target in unique_targets:
    target_texts = train_data[train_data['generalized_work_class'] == target]['work_name_corr']
    unique_texts = target_texts.unique().tolist()
    target_text_dict[target] = unique_texts

pprint(target_text_dict)

{'АКЗ мк': [' ак балок, стоек, пор',
            ' ак кабельное и технологисеской эстакады (цинотан)*',
            ' ак кабельное и технологической эстакады (политон-ур(уф))',
            ' ак кабельное и технологической эстакады (унипол ац)',
            ' т.1 ак надземной конструкции (ас1)',
            ' ак балок',
            ' т.1 ак надземной конструкции (ас)',
            ' ак кабельное и технологической эстакады (зачистка)',
            ' тк/межплощадочные трубопроводы от т.2 до пл. хал/огрунтовка и '
            'краска ак',
            ' ак и краска',
            ' тк/межплощадочные трубопроводы/огрунтовка и краска ак',
            ' опора тип 8 ак и краска надземной конструкции',
            ' ак и краска надземной конструкции',
            ' ак закладных деталей и перекрытия приямка',
            ' антикоррозийная защита балок и траверс',
            ' антикоррозийная защита стоек ограждения',
            ' антикоррозионная защита стоек ограждения',
            ' антикорро

In [97]:
def map_labels(text):
    '''Maps work_name in test into classes accured in train data'''
    for label, unique_texts in target_text_dict.items():
        if text in unique_texts:
            return label
    return None

In [98]:
test_data['predicted_labels'] = test_data['work_name_corr'].map(map_labels)

remain = test_data.loc[test_data['predicted_labels'].isna()]
print(f'Still unclassified in test data: {len(remain)} observations')

Still unclassified in test data: 4118 observations


In [99]:
bl = []
for i, row in test_data.iterrows():
    if row['predicted_labels']==row['generalized_work_class']:
        bl.append(True)
    else:
        bl.append(False)

In [89]:
test_data['true_pred'] = bl

In [90]:
# какие классы мы неправильно предсказали

test_data.loc[test_data['true_pred']==False, 'predicted_labels'].unique()

array(['Бетонирование', 'Монтаж вентиляции', None,
       'Монтаж кабель-каналов', 'АКЗ свай',
       'Монтаж мелких комплектующих кабеля', 'Монтаж опор (прочее)'],
      dtype=object)

## Accuracy: generalized_work_class

In [91]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score
test_data.fillna('', inplace=True)

print(accuracy_score(test_data.generalized_work_class.values, test_data.predicted_labels.values))

0.9771300946933331


In [102]:
test_data.loc[:, ['work_name', 'generalized_work_class', 'predicted_labels']]

,work_name,generalized_work_class,predicted_labels
0,"монтаж шаровых кранов, дроссельной шайбы, запо...",Монтаж мк,Монтаж мк
1,монтаж кипиа и зра с электроприводом,Монтаж ЗРА,Монтаж ЗРА
2,монтаж оборудования и приборов,Монтаж приборов,Монтаж приборов
3,тх./поз.2.13.1-2/монтаж арматуры,Монтаж арматуры,Монтаж арматуры
4,"монтаж шаровых кранов, огнепреградителя, дросс...",Монтаж мк,Монтаж мк
...,...,...,...
197901,электромонтажные работы,ПНР,ПНР
197902,гидроизоляция боковая обмазочная битумная,Гидроиспытания,Гидроиспытания
197903,"монтаж запорного клапана, кранов",Монтаж мк,Монтаж мк
197904,монтаж оборудования,Монтаж оборудования,Монтаж оборудования


In [46]:
train_data.to_csv('./Checkpoints/train_spell_correcred.csv')
test_data.to_csv('./Checkpoints/test_spell_correcred.csv')

# For global_work_class

For every GLOBAL_WORK_CLASS search for unique combination of 
    ```(work_name; generalized_work_class)```

In [54]:
test_data = pd.read_csv('./Checkpoints/test_spell_correcred.csv')
train_data = pd.read_csv('./Checkpoints/train_spell_correcred.csv')

In [103]:
# dictionary with unique 'global_work_class' values as keys
# and unique combinations of 'work_name' and 'generalized_work_class' as values
global_work_class_dict = {}
for index, row in train_data.iterrows():
    key = row['global_work_class']
    value = (row['work_name_corr'], row['generalized_work_class'])
    if key not in global_work_class_dict:
        global_work_class_dict[key] = set()
    global_work_class_dict[key].add(value)

In [104]:
global_work_class_pred = []

for index, row in test_data.iterrows():
    # Get the 'work_name' and 'generalized_work_class' from the test data
    work_name = row['work_name_corr']
    generalized_work_class = row['generalized_work_class']
    
    # Look up the values in the dictionary
    match_found = False
    for key, value_set in global_work_class_dict.items():
        if (work_name, generalized_work_class) in value_set:
            global_work_class_pred.append(key)
            match_found = True
            break

    # If no match is found, set 'global_work_class_pred' to None
    if not match_found:
        global_work_class_pred.append(None)

# Add 'global_work_class_pred' as a new column in the test data
test_data['global_work_class_pred'] = global_work_class_pred

In [106]:
bl = []
for i, row in test_data.iterrows():
    if row['global_work_class_pred']==row['global_work_class']:
        bl.append(True)
    else:
        bl.append(False)
test_data['true_pred_2'] = bl

In [108]:
# сколько классифицировалось верно и неверно

test_data.true_pred_2.value_counts()

true_pred_2
True     193780
False      4122
Name: count, dtype: int64

## Accuracy: global_work_class

In [109]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score
test_data.fillna('', inplace=True)

print(accuracy_score(test_data.global_work_class.values, test_data.global_work_class_pred.values))

0.9791715091307819


In [111]:
test_data.to_csv('Checkpoints/test_with_preds.csv')